### 專案名稱:  TQC+ 人工智慧：機器學習(Python 3)測驗
### 功能描述:  ML00205
### 版權所有: Dunk  
### 程式撰寫: Dunk 
### 參考網址:[【火炉炼AI】机器学习026-股票数据聚类分析-近邻传播算法](https://juejin.im/post/6844903671415701518 "")
### 撰寫日期：2020/10/06
### 改版日期:  2020/10/10 修改寫法
###                      2020/10/12 修改寫法
### 改版備註:  

題號：205 股票分群

難易度：易

（一）、 題目說明：

下載ML00205.zip解壓縮後，請開啟MLD02.py依下列題意進行組合及改寫，再將求取之答案依序輸入填答視窗中。
請注意，資料夾或程式碼中所提供的檔案路徑，不可進行變動。

（二）、 設計說明：

請使用affinity propagation分群（Clustering）撰寫程式，讀取symbol_map.json，這個資料包含所有股票代號以及對應名稱。
在特定時間內的某支股票市場變化情形則是以股票代號的csv的格式儲存。例如：AMZN.csv儲存Amazon在2003-2007的股價變動，其中包含序號、date（日期）、open（開盤價）、close（收盤價）等四個欄位。
請根據每日股價波動（收盤價-開盤價）找出公司行為的相似性，印出同屬一類的公司名稱。

（三）、 請依序回答下列問題：

請填入分群數量？
8
 此題您答對了，正確答案為：8

下列哪一選項的公司與Cisco同群，請填入選項？

(A) Dell, HP, IBM (B) General Dynamics, General Electrics, Goldman Sachs (C) ConocoPhillips, Chevron (D) CVS

A
 此題您答對了，正確答案為：A

下列哪一選項的公司與AIG同群，請填入選項？

(A) American express, Bank of America (B) Apple, Amazon, Yahoo (C) Comcast, Cablevision (D) Colgate-Palmolive, Kimberly-Clark

A
 此題您答對了，正確答案為：A

下列哪一選項的公司與Boeing同群，請填入選項？
(A) American express, Bank of America, DuPont de Nemours (B) GlaxoSmithKline, Home Depot, Kellogg (C) Home Depot, Kellogg (D) Canon, Caterpillar, Ford, Honda

D
 此題您答對了，正確答案為：D



In [49]:
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# TODO
symbol_file = 'symbol_map.json'

# TODO

symbols = pd.read_json(symbol_file, typ='series')
# symbols
columns = ['index_count','date','open','close','name']
df_source = pd.DataFrame()
for symbol in symbols.index:
# The daily fluctuations of the quotes 報價的每日波動
# TODO
    csv_file = symbol+".csv"
    df_temp = pd.read_csv(csv_file)
    df_temp['name'] = symbols.loc[symbol]    
    df_temp.columns = columns
    df_source = pd.concat([df_source,df_temp])
    
df_source['diff'] = df_source['close']-df_source['open']
df_source.drop(['open','close','index_count'],axis=1,inplace=True)

result_df=None 
#資料分群
for name, group in df_source[['date','diff']].groupby(df_source.name):    
    if result_df is None:
        result_df=group.rename(columns={'diff':name})
    else:
        result_df=pd.merge(result_df,\
                            group.rename(columns={'diff':name}),\
                            on='date',how='inner') 
    
result_df.drop(['date'],axis=1,inplace=True)
# DataFrame轉換為np.ndarray
stock_dataset=np.array(result_df).astype(np.float64)

# Standardize the data 標準化資料
# TODO
stock_dataset/=np.std(stock_dataset,axis=0)
selected_stocks = result_df.columns.tolist()

# Build a graph model from the correlations 根據相關性建立圖模型
# TODO
from sklearn.covariance import GraphicalLassoCV
edge_model=GraphicalLassoCV()

# Train the model 訓練模型
# TODO
from sklearn.cluster import affinity_propagation
edge_model.fit(stock_dataset)

# Build clustering model using affinity propagation 用相似性傳播構建分群模型
# TODO
_,labels=affinity_propagation(edge_model.covariance_)

n_labels = max(labels)
print("請填入分群數量？",n_labels+1)

# Print the results of clustering 列印分群結果
# TODO
for i in range(n_labels+1):    
    stocks=np.array(selected_stocks)[labels==i].tolist()
    print("Cluster", i+1, "-->",stocks)

#準備查詢資料    
df_query = pd.DataFrame([selected_stocks,labels]).T
df_query.drop(0,axis=1,inplace=True)
df_query.columns=['label']
df_query.index = selected_stocks
df_query


請填入分群數量？ 8
Cluster 1 --> ['Amazon', 'Apple', 'Yahoo']
Cluster 2 --> ['AIG', 'American express', 'Bank of America', 'DuPont de Nemours', 'General Dynamics', 'General Electrics', 'GlaxoSmithKline', 'Goldman Sachs', 'Home Depot', 'Kellogg']
Cluster 3 --> ['CVS']
Cluster 4 --> ['Cablevision', 'Comcast']
Cluster 5 --> ['Boeing', 'Canon', 'Caterpillar', 'Ford', 'Honda']
Cluster 6 --> ['Cisco', 'Dell', 'HP', 'IBM']
Cluster 7 --> ['Colgate-Palmolive', 'Kimberly-Clark']
Cluster 8 --> ['Chevron', 'ConocoPhillips']


,label
AIG,1
Amazon,0
American express,1
Apple,0
Bank of America,1
Boeing,4
CVS,2
Cablevision,3
Canon,4
Caterpillar,4


In [50]:
#儲存結果到csv
df_query.to_csv("result.csv")
target_col = 'label'
print("下列哪一選項的公司與Cisco同群，請填入選項？(計算出來是選項A)")
print(df_query.loc[['Cisco','Dell', 'HP', 'IBM'],target_col])
print(df_query.loc[['Cisco','General Dynamics', 'General Electrics', 'Goldman Sachs'],target_col])
print(df_query.loc[['Cisco','ConocoPhillips', 'Chevron'],target_col])
print(df_query.loc[['Cisco','CVS'],target_col])

下列哪一選項的公司與Cisco同群，請填入選項？(計算出來是選項A)
Cisco    5
Dell     5
HP       5
IBM      5
Name: label, dtype: object
Cisco                5
General Dynamics     1
General Electrics    1
Goldman Sachs        1
Name: label, dtype: object
Cisco             5
ConocoPhillips    7
Chevron           7
Name: label, dtype: object
Cisco    5
CVS      2
Name: label, dtype: object


In [51]:
print("下列哪一選項的公司與AIG同群，請填入選項？(計算出來是選項A)")
print(df_query.loc[[ 'AIG','American express', 'Bank of America'],target_col])
print(df_query.loc[[ 'AIG','Apple', 'Amazon', 'Yahoo'],target_col])
print(df_query.loc[[ 'AIG','Comcast', 'Cablevision'],target_col])
print(df_query.loc[[ 'AIG','Colgate-Palmolive', 'Kimberly-Clark'],target_col])

下列哪一選項的公司與AIG同群，請填入選項？(計算出來是選項A)
AIG                 1
American express    1
Bank of America     1
Name: label, dtype: object
AIG       1
Apple     0
Amazon    0
Yahoo     0
Name: label, dtype: object
AIG            1
Comcast        3
Cablevision    3
Name: label, dtype: object
AIG                  1
Colgate-Palmolive    6
Kimberly-Clark       6
Name: label, dtype: object


In [52]:
print("下列哪一選項的公司與Boeing同群，請填入選項？(計算出來是選項D)")
print(df_query.loc[[ 'Boeing','American express', 'Bank of America'],target_col])
print(df_query.loc[[ 'Boeing','GlaxoSmithKline', 'Home Depot', 'Kellogg'],target_col])
print(df_query.loc[[ 'Boeing','Home Depot', 'Kellogg'],target_col])
print(df_query.loc[[ 'Boeing','Canon', 'Caterpillar', 'Ford', 'Honda'],target_col])

下列哪一選項的公司與Boeing同群，請填入選項？(計算出來是選項D)
Boeing              4
American express    1
Bank of America     1
Name: label, dtype: object
Boeing             4
GlaxoSmithKline    1
Home Depot         1
Kellogg            1
Name: label, dtype: object
Boeing        4
Home Depot    1
Kellogg       1
Name: label, dtype: object
Boeing         4
Canon          4
Caterpillar    4
Ford           4
Honda          4
Name: label, dtype: object
